In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## EDA

In [ ]:
df = pd.read_csv('dataset.csv')
df.sample(5)

In [ ]:
df.info()

In [ ]:
def plot_products(df):
    product_counts = df["nombre_producto"].value_counts()
    print(f"Total number of products is: {len(product_counts)}")

    plt.figure(figsize=(12, 6))
    product_counts.plot(kind='bar')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.xlabel("Nombre del Producto")
    plt.ylabel("Cantidad")
    plt.title("Productos de tienda Dummy")
    plt.show()

def plot_products_categories(df):
    product_counts = df["categoria"].value_counts()
    print(f"Total number of products categories is: {len(product_counts)}")

    plt.figure(figsize=(12, 6))
    product_counts.plot(kind='bar')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.xlabel("Categoría")
    plt.ylabel("Cantidad")
    plt.title("Categorías de productos de tienda Dummy")
    plt.show()

In [ ]:
plot_products(df)
plot_products_categories(df)

Por cantidad no se refiere a stock total, sino las veces que aparece en el .csv dicho producto (como fila).

5 categorías, especial incapie en deportivas y botas.

In [ ]:
df.describe()

In [ ]:
df = df.drop(columns=["producto_id"])

No se puede ver la evolución del stock, ni de las unidades vendidas ni del precio. Vamos a tirar por ahí.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_stock_variations(df, productos=None):
    df["fecha"] = pd.to_datetime(df["fecha"])

    if productos is None:
        productos_disponibles = df["nombre_producto"].unique()
        raise ValueError(f"No se especificaron productos.\nProductos disponibles: {productos_disponibles}")

    if isinstance(productos, str):
        productos = [productos]

    for producto in productos:
        df_prod = df[df["nombre_producto"] == producto]
        
        # Agrupamos por fecha para asegurarnos de no tener múltiples valores por mes
        df_prod = df_prod.groupby("fecha")[["stock_inicial", "stock_final"]].sum().reset_index()

        plt.figure(figsize=(12, 5))
        plt.plot(df_prod["fecha"], df_prod["stock_inicial"], label="Stock Inicial", linestyle="--")
        plt.plot(df_prod["fecha"], df_prod["stock_final"], label="Stock Final", linestyle="-")

        plt.title(f"Evolución del Stock - {producto}")
        plt.xlabel("Fecha")
        plt.ylabel("Unidades en Stock")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [ ]:
plot_stock_variations(df, productos=list(df["nombre_producto"].unique()))

Al ser un dataset sintético, no vemos problemas. Casi siempre hay un excedente de stock, pero podemos ver en qué meses y para qué productos hemos vendido todo el stock en dicho mes.

## Correlaciones

Tendríamos que ver las correlaciones que hay entre ciertas variables.

In [ ]:
def plot_correlations(df):   
    numerical_variables = df.select_dtypes(include=["number"]).columns.tolist()
    # Calculate correlation matrices for train_data and test_data
    corr_train = df[numerical_variables].corr()
    
    # Create masks for the upper triangle
    mask_train = np.triu(np.ones_like(corr_train, dtype=bool))
    
    # Set the text size and rotation
    annot_kws = {"size": 8, "rotation": 45}
    
    # Generate heatmaps for train_data
    plt.figure(figsize=(15, 8))
    plt.subplot(1, 2, 1)
    ax_train = sns.heatmap(corr_train, mask=mask_train, cmap='viridis', annot=True,
                          square=True, linewidths=.5, xticklabels=1, yticklabels=1, annot_kws=annot_kws)
    plt.title('Correlation Heatmap - Train Data')
    
    # Adjust layout
    plt.tight_layout()
    
    # Show the plots
    plt.show()

In [ ]:
plot_correlations(df)

Podemos sacar conclusiones lógicas:

1. Si el stock final es grande, las unidades vendidas son bajas, por lo que se estima que no hace falta renovar en exceso el stock
2. Si no se ha repuesto este mes, significa que las unidades vendidas no han sido muy altas
3. Existe una relación curiosa con stock_final, reposicion_este_mes y stock_inicial. Correlación positiva.
4. Si hay una promoción o un evento especial (asumiendo que este sea positivo para el negocio), las ventas crecen.
5. Si te has quedado sin stock durante un mes, es más probable que repongas para el próximo.altas

**IMPORTANTE**

Nuestra `Y_pred` = reponer_proximo_mes

## Estacionalidad

Es un modelo de predicción, es importante discernir entre estaciones del año. Tenemos:
- Invierno (Diciembre-Febrero)
- Primavera (Marzo-Mayo)
- Verano (Junio-Agosto)
- Otoño (Septiembre-Noviembre)

In [ ]:
stations = {
    "winter": [12, 1, 2], 
    "spring": [3, 4, 5], 
    "summer": [6, 7, 8], 
    "autumn": [9, 10, 11]}

def add_stations(df):
    def get_station(mes):
        for estacion, meses in stations.items():
            if mes in meses:
                return estacion
        return "desconocido" # empty value

    df["estacion"] = df["mes"].apply(get_station)
    return df

In [ ]:
df = add_stations(df)

In [ ]:
df.sample(5)

Importante destacar que el modelo no entiende caracteres, sino números. Tenemos que hacer un `encoding`.

- OneHot-encoding: es lo que tenemos que hacer
- Label Encoding: puede ir bien, pero puede añadir un orden artificial. No interesa

In [ ]:
def convert_into_onehot(df, columns):
    dummies = pd.get_dummies(df[columns], prefix=columns).astype(int)
    df = df.drop(columns=columns)
    df = df.join(dummies)
    return df

In [ ]:
df = convert_into_onehot(df, ["estacion"])

In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"])
# se podría hacer day_of_the_week, is_weekend, trimetre... pero nuestro dataset no tiene esos datos
df = df.drop(columns=["fecha"])

In [ ]:
df.sample(5)

In [ ]:
plot_correlations(df)

## Procesamiento de los productos 

De la misma manera, tenemos que convertir los productos y sus categorias con el One Hot Encoding.

In [ ]:
df = convert_into_onehot(df, ["nombre_producto", "categoria"])

In [ ]:
df.sample(5)

## Modelo Predictivo